In [ ]:
import xarray as xr
import numpy as np
%matplotlib inline

In [ ]:
dsi = xr.open_dataset('era5_land_invariant.nc')
dsi

In [ ]:
dsif = dsi.stack(points=('latitude', 'longitude')).reset_index(('time', 'points'))

In [ ]:
dsif

In [ ]:
ds = xr.open_dataset('era5_land_monthly_t2m_1981-2018.nc')
ds

In [ ]:
dsifs = dsif.where(np.isfinite(ds.t2m.isel(time=[0]).stack(points=('latitude', 'longitude')).reset_index(('time', 'points'))), drop=True)

In [ ]:
dsifs

In [ ]:
dsifs.to_netcdf('era5_land_invariant_flat.nc', encoding={'z':dsi.z.encoding, 'lsm':dsi.lsm.encoding})

In [ ]:
for t in ds.time.values:
    sel = ds.sel(time=[t])
    sel = sel.stack(points=('latitude', 'longitude')).reset_index(('time', 'points'))
    sel = sel.where(np.isfinite(sel.t2m), drop=True)
    sel.to_netcdf('tmp_{}.nc'.format(str(t)[:10]))

In [ ]:
dso = xr.open_mfdataset('tmp_*.nc', concat_dim='time', combine='nested').rename_vars({'time_':'time'})

In [ ]:
dso.to_netcdf('era5_land_monthly_t2m_1981-2018_flat.nc', encoding={'t2m':ds.t2m.encoding})

In [ ]:
%rm tmp_*

In [ ]:
ds = xr.open_dataset('era5_land_monthly_prcp_1981-2018.nc')
ds

In [ ]:
for t in ds.time.values:
    sel = ds.sel(time=[t])
    sel = sel.stack(points=('latitude', 'longitude')).reset_index(('time', 'points'))
    sel = sel.where(np.isfinite(sel.tp), drop=True)
    sel.to_netcdf('tmp_{}.nc'.format(str(t)[:10]))

In [ ]:
dso = xr.open_mfdataset('tmp_*.nc', concat_dim='time', combine='nested').rename_vars({'time_':'time'})

In [ ]:
dso.to_netcdf('era5_land_monthly_prcp_1981-2018_flat.nc', encoding={'tp':ds.tp.encoding})

In [ ]:
%rm tmp_*